# Aprenentatge Supervisat - Regressions

**Descripció**

Anem a practicar i a familiaritzar-nos amb regressions

In [5]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor


### Carreguem les dades

In [2]:
dfdelays = pd.read_pickle('../data/S11-CleanDelayedFlights.pickle')

### Preprocés

In [3]:
dfdelays.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,ArrivalTime_month_sin,ArrivalTime_month_cos,CRSArrivalTime_hour_sin,CRSArrivalTime_hour_cos,CRSArrivalTime_day_sin,CRSArrivalTime_day_cos,CRSArrivalTime_dayofweek_sin,CRSArrivalTime_dayofweek_cos,CRSArrivalTime_month_sin,CRSArrivalTime_month_cos
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,0.5,0.866025,-0.500000,0.866025,0.571268,0.820763,0.433884,-0.900969,0.5,0.866025
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,0.5,0.866025,0.500000,-0.866025,0.571268,0.820763,0.433884,-0.900969,0.5,0.866025
2,2008,1,3,4,628.0,620,804.0,750,WN,448,...,0.5,0.866025,0.965926,-0.258819,0.571268,0.820763,0.433884,-0.900969,0.5,0.866025
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,...,0.5,0.866025,-0.965926,0.258819,0.571268,0.820763,0.433884,-0.900969,0.5,0.866025
5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,378,...,0.5,0.866025,-0.707107,0.707107,0.571268,0.820763,0.433884,-0.900969,0.5,0.866025


In [4]:
dfdelays.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928371 entries, 0 to 7009727
Data columns (total 87 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   Year                            int64         
 1   Month                           int64         
 2   DayofMonth                      int64         
 3   DayOfWeek                       int64         
 4   DepTime                         object        
 5   CRSDepTime                      object        
 6   ArrTime                         object        
 7   CRSArrTime                      int64         
 8   UniqueCarrier                   object        
 9   FlightNum                       int64         
 10  TailNum                         object        
 11  ActualElapsedTime               float64       
 12  CRSElapsedTime                  float64       
 13  AirTime                         float64       
 14  ArrDelay                        float64       
 15

### Generem el train_test

## Nivell 1

### Exercici 1
Crea almenys tres models de regressió diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv.

Crearem aquests models:
- LinearRegression
- RigdeCV
- HuberRegressor (ourlier-robust)
- GammaRegressor
- SVR
- DecisionTreeRegressor
- RandomForestRegressor
- GradientBoostingRegressor
- MLPRegressor
- XGBoostRegressor


In [ ]:
from sklearn.linear_model import LinearRegression, RidgeCV, HuberRegressor, GammaRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from xgboost import XGBRegressor

### Exercici 2
Compara’ls en base al MSE i al R2 .

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score


### Exercici 3
Entrena’ls utilitzant els diferents paràmetres que admeten.

### Exercici 4
Compara el seu rendiment utilitzant l’aproximació traint/test o utilitzant totes les dades (validació interna)

#### CV

In [ ]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

## Nivell 2
### Exercici 5
Realitza algun procés d’enginyeria de variables per millorar-ne la predicció

## Nivell 3
### Exercici 6
No utilitzis la variable DepDelay a l’hora de fer prediccions